In [1]:
import sys

sys.path.insert(0,'..')
#!pip install openpyxl

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Load excel

In [5]:
df = pd.read_excel("../data/RB_TC_extract_2022_07_07.xlsx")
df = df.iloc[1: , :] #first line are units

In [ ]:
Short magnet ID	Manufacturer	Inner cable type	Outer cable type	I_Q_SM18

In [16]:
df = pd.read_excel("../data/RB_TC_extract_2022_07_07.xlsx", "magnet data")

df.dropna(how="all").to_csv("mb_features.csv")

In [18]:
df.columns

Index(['Short magnet ID', 'Manufacturer', 'Inner cable type',
       'Outer cable type', 'I_Q_SM18', 'Renamed short magnet ID',
       'MTF Status'],
      dtype='object')

# Look at unique values

In [ ]:
df.columns.values

In [ ]:
df['FPA Reason'].value_counts()

In [ ]:
df['Type of Quench'].value_counts()

In [ ]:
df['Nr in Q event'].value_counts()

In [ ]:
df['QDS trigger origin'].value_counts() 

In [ ]:
df['Position'].value_counts() 

In [ ]:
df['Analysis performed by'].value_counts()

In [ ]:
df['Comment'].value_counts()[:20]

# Histograms

In [ ]:
df["Date (FGC)"] = pd.to_datetime(df["Date (FGC)"])
df["year"] = df["Date (FGC)"].dt.year.dropna()
df["month"] = df["Date (FGC)"].dt.month.dropna()

In [ ]:
def plot_hist(df, column, ax, bins=None, title=None, range=None, em=True, xlim=None, ylog=False):
    df[column].hist(label="all quenches", ax=ax, bins=bins, align='left', rwidth=0.9, range=range)
    df[df['Type of Quench'] =="GHe propagation"][column].hist(label="secondary quenches (GHe)", ax=ax, bins=bins, rwidth=0.9, align='left', range=range)
    df[df['Nr in Q event'] == 1][column].hist(label="primary quenches", ax=ax, bins=bins, rwidth=0.9, align='left', range=range)
    if em:
        df[df['Type of Quench'] =="EM disturbance"][column].hist(label="secondary quenches (EM)", ax=ax, bins=bins, rwidth=0.9, align='left', range=range)
    if xlim:
        ax.set_xlim(xlim)
    if ylog:
        ax.set_yscale("log")
    ax.legend()
    ax.set_title(title)

In [ ]:
fig, ax = plt.subplots(5,3, figsize=(25,30))
plot_hist(df, column="year", ax=ax[0,0], bins=np.arange(2008, 2023), title="Year of Quench")
plot_hist(df, column="Circuit Name", ax=ax[0,1], bins=8, title="Circuit of Quench")
plot_hist(df, column="I_Q_M", ax=ax[0,2], title="Current of Quench", range=(df["I_Q_M"].min(), df["I_Q_M"].max()))
plot_hist(df, column="I_Q_SM18", ax=ax[1,0], title="SM18 Test Current of Quenched Magnet", range=(df["I_Q_SM18"].min(), df["I_Q_SM18"].max()))
plot_hist(df, column="Short magnet ID", ax=ax[1,1], bins=100, title="Short magnet ID", range=(df["Short magnet ID"].min(), df["Short magnet ID"].max()))
plot_hist(df, column="Manufacturer", ax=ax[1,2], bins=3, title="Manufacturer")
plot_hist(df[df["Outer cable type"].apply(lambda x: isinstance(x, str))], column="Outer cable type", ax=ax[2,0], title="Outer cable type", bins=df[df["Outer cable type"].apply(lambda x: isinstance(x, str))]["Outer cable type"].unique())
plot_hist(df, column="Inner cable type", ax=ax[2,1], title="Inner cable type", em=False)

plot_hist(df, column="V_symm_max", ax=ax[2,2], title="V_symm_max", xlim=(0,2), bins=100, range=(df["V_symm_max"].min(), df["V_symm_max"].max()))
plot_hist(df.drop(df[df["dU_iQPS/dt"].apply(lambda x: isinstance(x, str))].index), column="dU_iQPS/dt", ax=ax[3,0], title="dU_iQPS/dt", xlim=(-25,25), bins=100, range=(df.drop(df[df["dU_iQPS/dt"].apply(lambda x: isinstance(x, str))].index)["dU_iQPS/dt"].min(), df.drop(df[df["dU_iQPS/dt"].apply(lambda x: isinstance(x, str))].index)["dU_iQPS/dt"].max()))

plot_hist(df.drop(df[df["Quench origin"]=="?"].index), column="Quench origin", ax=ax[3,1], title="Quench origin")
plot_hist(df.drop(df[df["R_DL_max"].apply(lambda x: isinstance(x, str))].index), column="R_DL_max", ax=ax[4,0], title="R_DL_max", xlim=(-5,200), bins=300, range=(df.drop(df[df["R_DL_max"].apply(lambda x: isinstance(x, str))].index)["R_DL_max"].min(), df.drop(df[df["R_DL_max"].apply(lambda x: isinstance(x, str))].index)["R_DL_max"].max()))
plot_hist(df, column="I at R_DL_max", ax=ax[4,1], title="I at R_DL_max", ylog=True, bins=200, range=(df["I at R_DL_max"].min(), df["I at R_DL_max"].max()))